In [ ]:
!pip install flask pyngrok transformers torch newspaper3k lxml[html5] lxml_html_clean


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.9 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=02947ca1b018be9c5fba676f440ce51a10cf0ac9baeb3da0ff07f635087c546c
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=bc65dc18dedc7cf1608a44020ab80d59c8d86f0c7eeff81537fe17030bf56369
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

In [ ]:
!mkdir -p templates
!mkdir -p static
!mkdir -p uploads


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request
from transformers import pipeline
from newspaper import Article
import os

app = Flask(__name__)

# ------------------- Load Summarization Model -------------------
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_text_or_url(input_text=None, input_url=None):
    if input_url:
        article = Article(input_url)
        article.download()
        article.parse()
        text = article.text
    elif input_text:
        text = input_text
    else:
        return "❗ Please provide text or a URL."

    summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
    return summary[0]['summary_text']


@app.route("/", methods=["GET", "POST"])
def home():
    summary = ""
    input_text = ""
    input_url = ""

    if request.method == "POST":
        input_text = request.form.get("input_text")
        input_url = request.form.get("input_url")

        summary = summarize_text_or_url(input_text=input_text, input_url=input_url)

    return render_template("index.html", summary=summary, input_text=input_text, input_url=input_url)


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)



Writing app.py


# HTML

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>📰 Text & URL Summarizer</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
<div class="container">
    <h1>📰 Text & URL Summarizer</h1>

    <div class="card">
        <h2>Enter Text</h2>
        <form method="post">
            <textarea name="input_text" placeholder="Paste text here..." rows="6">{{ input_text }}</textarea>
            <h3>OR</h3>
            <input type="url" name="input_url" placeholder="Enter article URL" value="{{ input_url }}">
            <button type="submit">Summarize 🚀</button>
        </form>
    </div>

    {% if summary %}
    <div class="card">
        <h2>Summary</h2>
        <p>{{ summary }}</p>
    </div>
    {% endif %}
</div>
</body>
</html>

Writing templates/index.html


In [ ]:
%%writefile static/style.css
body {
    font-family: 'Segoe UI', sans-serif;
    background: linear-gradient(135deg, #0f2027, #203a43, #2c5364);
    color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100vh;
    margin: 0;
}
.container {
    text-align: center;
    width: 60%;
}
h1, h2, h3 {
    margin-bottom: 15px;
}
.card {
    background: rgba(255, 255, 255, 0.1);
    padding: 20px;
    border-radius: 12px;
    box-shadow: 0 0 15px rgba(0,0,0,0.4);
    margin-bottom: 20px;
}
textarea, input, button {
    width: 90%;
    margin: 10px 0;
    padding: 10px;
    border-radius: 8px;
    border: none;
}
textarea {
    resize: vertical;
}
button {
    background: #FFD700;
    cursor: pointer;
    font-weight: bold;
}
button:hover {
    background: #FFA500;
}

Writing static/style.css


In [ ]:
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

^C
^C


In [ ]:
!lsof -i : 8000


lsof: unacceptable port specification in: -i :
lsof 4.93.2
 latest revision: https://github.com/lsof-org/lsof
 latest FAQ: https://github.com/lsof-org/lsof/blob/master/00FAQ
 latest (non-formatted) man page: https://github.com/lsof-org/lsof/blob/master/Lsof.8
 usage: [-?abhKlnNoOPRtUvVX] [+|-c c] [+|-d s] [+D D] [+|-E] [+|-e s] [+|-f[gG]]
 [-F [f]] [-g [s]] [-i [i]] [+|-L [l]] [+m [m]] [+|-M] [-o [o]] [-p s]
 [+|-r [t]] [-s [p:s]] [-S [t]] [-T [t]] [-u s] [+|-w] [-x [fl]] [--] [names]
Use the ``-h'' option to get more help information.


In [ ]:
!kill -9 1144


/bin/bash: line 1: kill: (1144) - No such process


In [ ]:
!nohup python app.py > flask.log 2>&1 &

In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "32rscLFLHDJnxKCvvA7VUhgIwAu_36nxu7tTVxR6mfaSa7hdB"
public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)


🌍 Public URL: NgrokTunnel: "https://2b7159dd5d72.ngrok-free.app" -> "http://localhost:8000"
